In [1]:
'''
Authors: Daniel M. Low
License: See license in github repository
'''

import os
import pandas as pd
import numpy as np
import datetime
ts = datetime.datetime.utcnow().strftime('%y-%m-%dT%H-%M-%S')

pd.set_option("display.max_columns", None)
# pd.options.display.width = 0

on_colab = True

if on_colab:
  from google.colab import drive
  drive.mount('/content/drive')
  input_dir = f'/content/drive/MyDrive/psy2085/labs/9_zero_shot_classification/'
  output_dir = f'/content/drive/MyDrive/psy2085/labs/9_zero_shot_classification/'
else:
  input_dir = './data/input/'
  output_dir = './data/output/'

Mounted at /content/drive


# SEANCE lexicon

Based on General Inquirer, Laswell, and NRC lexicons



In [2]:
with open(input_dir+'inquirerbasic.txt', 'r') as f:
    seance_txt = f.readlines()

seance = {}
cat_names = []
for cat in seance_txt:
    cat =cat.strip('\n')
    cat_items = cat.split('\t')
    seance[cat_items[0]]=cat_items[1:]
    cat_names.append(cat_items[0])


In [3]:
seance.keys()

dict_keys(['Positiv_GI', 'Negativ_GI', 'Pstv_GI', 'Affil_GI', 'Ngtv_GI', 'Hostile_GI', 'Strong_GI', 'Power_GI', 'Weak_GI', 'Submit_GI', 'Active_GI', 'Passive_GI', 'Pleasur_GI', 'Pain_GI', 'Feel_GI', 'Arousal_GI', 'Emot_GI', 'Virtue_GI', 'Vice_GI', 'Ovrst_GI', 'Undrst_GI', 'Academ_GI', 'Doctrin_GI', 'Econ_2_GI', 'Exch_GI', 'Econ_GI', 'Exprsv_GI', 'Legal_GI', 'Milit_GI', 'Polit_2_GI', 'Polit_GI', 'Relig_GI', 'Role_GI', 'Coll_GI', 'Work_GI', 'Ritual_GI', 'Socrel_GI', 'Race_GI', 'Kin_2_GI', 'Male_GI', 'Female_GI', 'Nonadlt_GI', 'Hu_GI', 'Ani_GI', 'Place_GI', 'Social_GI', 'Region_GI', 'Route_GI', 'Aquatic_GI', 'Land_GI', 'Sky_GI', 'Object_GI', 'Tool_GI', 'Food_GI', 'Vehicle_GI', 'Bldgpt_GI', 'Comnobj_GI', 'Natobj_GI', 'Bodypt_GI', 'Comform_GI', 'Com_GI', 'Say_GI', 'Need_GI', 'Goal_GI', 'Try_GI', 'Means_GI', 'Persist_GI', 'Complet_GI', 'Fail_GI', 'Natrpro_GI', 'Begin_GI', 'Vary_GI', 'Increas_GI', 'Decreas_GI', 'Finish_GI', 'Stay_GI', 'Rise_GI', 'Exert_GI', 'Fetch_GI', 'Travel_GI', 'Fall_GI',

In [10]:
list(seance['Anger_NRC'])

['expletive',
 'inept',
 'unfulfilled',
 'lynch',
 'agitation',
 'wrongdoing',
 'satanic',
 'militia',
 'bellows',
 'warden',
 'complicate',
 'opera',
 'death',
 'shoplifting',
 'excitation',
 'frenetic',
 'unlawful',
 'stifled',
 'miserable',
 'penalty',
 'obliterated',
 'reproach',
 'grated',
 'offender',
 'devastation',
 'wring',
 'dissension',
 'condemn',
 'explode',
 'thoughtless',
 'sting',
 'fleece',
 'vengeful',
 'segregate',
 'subsidy',
 'abomination',
 'revolting',
 'exaggerate',
 'hellish',
 'disrespect',
 'delay',
 'delinquent',
 'crusade',
 'disgusting',
 'disobedient',
 'duel',
 'annihilate',
 'antisocial',
 'oppression',
 'banger',
 'grumpy',
 'obliterate',
 'inequality',
 'dumps',
 'tree',
 'dismay',
 'mortality',
 'defy',
 'disease',
 'harassing',
 'accused',
 'bad',
 'skirmish',
 'gall',
 'combatant',
 'hot',
 'horrible',
 'frightful',
 'unfair',
 'tolerate',
 'coercion',
 'backbone',
 'adversary',
 'misconception',
 'thump',
 'criminal',
 'devastating',
 'berserk',
 

In [5]:
# Clean seance names we created
clean_names = pd.read_csv(input_dir+'clean_seance_names.csv', index_col = 0)
clean_names

,lexicon_name,clean_name,category,representative_example,notes
0,Positiv_GI,positive_gi,sentiment,"happy, altruistic, amazement",NaN
1,Negativ_GI,negative,sentiment,"sad, burden, helplessness",NaN
2,Pstv_GI,positive_gi_2,sentiment,"calm, friend, rich",osgood
3,Affil_GI,affiliation_gi,social,"charm, appeal, friendship",NaN
4,Ngtv_GI,negative_gi_2,sentiment,"bitter, cruel, horrible",earlier version of Negativ
...,...,...,...,...,...
187,Negative_NRC,negative_nrc,emotion,"arrogant, kidnap, lack",NaN
188,Positive_NRC,positive_nrc,emotion,"greeting, priest, hire",NaN
189,Sadness_NRC,sadness_nrc,emotion,"confession, mournful, lament",NaN
190,Surprise_NRC,surprise_nrc,emotion,"opera, clown, shock",NaN


In [6]:
clean_names_d = dict(zip(clean_names['lexicon_name'], clean_names['clean_name']))
examples_d = dict(zip(clean_names['lexicon_name'], clean_names['representative_example']))
print(clean_names_d.get('Affil_GI'))
print(examples_d.get('Affil_GI'))


affiliation_gi
charm, appeal, friendship


In [ ]:
docs_lower = [n.lower() for n in docs]

In [ ]:

import pandas as pd
import re
from collections import Counter

# from text.utils.count_words import word_count
import numpy as np

import string

def word_count(docs, return_zero = []):

	'''

	Args:
		docs:
		return_zero: list containing docs not to count, default = None.
			tokens

	Returns:

	'''

	# initializing string
	# test_string = "Geeksforgeeks,    is best @# Computer Science Portal.!!!"
	word_counts = []
	for doc_i in docs:
		doc_i = str(doc_i)
		if doc_i in return_zero or len(doc_i)==0:
			word_counts.append(0)
		else:
			# after asserting that len(doc_i)>0
			if len(doc_i)>0 and sum([i.strip(string.punctuation).isalpha() for i in doc_i.split()]) == 0:
				# 'R.I.P.'
				word_counts.append(1)
			else:
				wc_i = sum([i.strip(string.punctuation).isalpha() for i in doc_i.split()])
				word_counts.append(wc_i)
	if len(word_counts)==1:
		return word_counts[0]
	else:

		return word_counts

def count_lexicons_in_doc(doc,tokens=[], return_zero = [], return_matches=False):
	# TODO, perhaps return match and context (3 words before and after)
	'''

	Args:
		doc:
		tokens: lexicon tokens
		return_zero:
		normalize:
		return_matches:

	Returns:

	'''

	# remove punctuation except apostrophes because we need to search for things like "don't want to live"
	text = re.sub("[^\w\d'\s]+",'',doc.lower())
	counter = 0
	matched_tokens = []
	for token in tokens:
		token = token.lower()
		matches = text.count(token)
		counter += matches
		if return_matches and matches>0:
			# print(matches, token)
			matched_tokens.append(token)



	if return_matches:
		return counter, matched_tokens
	else:
		return counter

def extract(docs,lexicons, normalize = True, return_zero =[], return_matches=False, add_word_count = True):
	# TODO: return zero is for entire docs, shouldnt it be for tokens?
	'''

	Args:
		docs:
		lexicons:
		normalize:
			divide by zero
		return_zero:

	Returns:

	'''
	#process all posts
	# docs is list of list
	# lexicons is dictionary {'category':[token1, token2, ...], 'category2':[]}
	docs = [doc.replace('\n', ' ').replace('  ',' ').replace('“', '').replace('”', '') for doc in docs]

	# feature_names = list(lexicons.keys())
	# full_column_names = list(df_subreddit.columns) + feature_names
	# subreddit_features = pd.DataFrame(columns=full_column_names)

	# word_counts = reddit_data.n_words.values
	# all words in subgroup

	feature_vectors = {}
	matches = {}

	for category in list(lexicons.keys()):

		lexicon_tokens = lexicons.get(category)
		if return_matches:
			counts_and_matched_tokens = [count_lexicons_in_doc(doc,tokens=lexicon_tokens,  return_zero = return_zero, return_matches=return_matches) for doc in docs]
			counts =  [n[0] for n in counts_and_matched_tokens]
			matched_tokens =  [n[1] for n in counts_and_matched_tokens if n[1]!=[]]
			matches[category] = matched_tokens



		else:
			counts = [count_lexicons_in_doc(doc,tokens=lexicon_tokens, return_zero = return_zero, return_matches=return_matches) for doc in docs]
		# one_category = one_category/word_counts #normalize

		feature_vectors[category]=counts


		# # feature_vector = extract_NLP_features(post, features) #removed feature_names from output
		# if len(feature_vector) != 0:
		#     raw_series = list(df_subreddit.iloc[pi])
		#     subreddit_features = subreddit_features.append(pd.Series(raw_series + feature_vector, index=full_column_names), ignore_index=True)

	# feature_vectors0   = pd.DataFrame(docs, columns = ['docs'])
	# feature_vectors = pd.concat([feature_vectors0,pd.DataFrame(feature_vectors)],axis=1)
	feature_vectors   = pd.DataFrame(feature_vectors)

	#     feature_vectors   = pd.DataFrame(docs)
	#     feature_vectors['docs']=docs

	if normalize:

		wc = word_count(docs, return_zero=return_zero)
		wc = np.array(wc)
		feature_vectors_normalized = np.divide(feature_vectors.values.T,wc).T
		feature_vectors = pd.DataFrame(feature_vectors_normalized, index = feature_vectors.index, columns=feature_vectors.columns)

	if add_word_count and normalize:
		feature_vectors['word_count'] = wc
	elif add_word_count and not normalize:
		wc = word_count(docs, return_zero=return_zero)
		feature_vectors['word_count'] = wc





		# feature_vectors = feature_vectors/wc

	if return_matches:
		# all lexicons
		matches_counter_d = {}
		for lexicon_name_i in list(lexicons.keys()):
			if matches.get(lexicon_name_i):
				x = Counter([n for i in matches.get(lexicon_name_i) for n in i])
				matches_counter_d[lexicon_name_i]= matches_counter_d[lexicon_name_i]={k: v for k, v in sorted(x.items(), key=lambda item: item[1])}
		# Counter([n for i in matches.get(lexicon_name_i) for n in i]) for lexicon_name_i in lexicons_d.keys()]




		return feature_vectors, matches_counter_d
	else:
		return feature_vectors


In [ ]:

return_matches = True
normalize = False # if True, divide by word count (generally a good idea, an appearance of loneliness in a short doc weighs more than in a long doc)
features, matches = extract(docs,seance, normalize = normalize, return_matches=return_matches)
features

,Positiv_GI,Negativ_GI,Pstv_GI,Affil_GI,Ngtv_GI,Hostile_GI,Strong_GI,Power_GI,Weak_GI,Submit_GI,Active_GI,Passive_GI,Pleasur_GI,Pain_GI,Feel_GI,Arousal_GI,Emot_GI,Virtue_GI,Vice_GI,Ovrst_GI,Undrst_GI,Academ_GI,Doctrin_GI,Econ_2_GI,Exch_GI,Econ_GI,Exprsv_GI,Legal_GI,Milit_GI,Polit_2_GI,Polit_GI,Relig_GI,Role_GI,Coll_GI,Work_GI,Ritual_GI,Socrel_GI,Race_GI,Kin_2_GI,Male_GI,Female_GI,Nonadlt_GI,Hu_GI,Ani_GI,Place_GI,Social_GI,Region_GI,Route_GI,Aquatic_GI,Land_GI,Sky_GI,Object_GI,Tool_GI,Food_GI,Vehicle_GI,Bldgpt_GI,Comnobj_GI,Natobj_GI,Bodypt_GI,Comform_GI,Com_GI,Say_GI,Need_GI,Goal_GI,Try_GI,Means_GI,Persist_GI,Complet_GI,Fail_GI,Natrpro_GI,Begin_GI,Vary_GI,Increas_GI,Decreas_GI,Finish_GI,Stay_GI,Rise_GI,Exert_GI,Fetch_GI,Travel_GI,Fall_GI,Think_GI,Know_GI,Causal_GI,Ought_GI,Perceiv_GI,Compare_GI,Eval_2_GI,Eval_GI,Solve_GI,Abs_2_GI,Abs_GI,Quality_GI,Quan_GI,Numb_GI,Ord_GI,Card_GI,Freq_GI,Dist_GI,Time_2_GI,Time_GI,Space_GI,Pos_GI,Dim_GI,Rel_GI,Color_GI,Self_GI,Our_GI,You_GI,Name_GI,Yes_GI,No_GI,Negate_GI,Intrj_GI,Iav_GI,Dav_GI,Sv_GI,Ipadj_GI,Indadj_GI,Powgain_Lasswell,Powloss_Lasswell,Powends_Lasswell,Powaren_Lasswell,Powcon_Lasswell,Powcoop_Lasswell,Powaupt_Lasswell,Powpt_Lasswell,Powdoct_Lasswell,Powauth_Lasswell,Powoth_Lasswell,Powtot_Lasswell,Rcethic_Lasswell,Rcrelig_Lasswell,Rcgain_Lasswell,Rcloss_Lasswell,Rcends_Lasswell,Rctot_Lasswell,Rspgain_Lasswell,Rsploss_Lasswell,Rspoth_Lasswell,Rsptot_Lasswell,Affgain_Lasswell,Affloss_Lasswell,Affpt_Lasswell,Affoth_Lasswell,Afftot_Lasswell,Wltpt_Lasswell,Wlttran_Lasswell,Wltoth_Lasswell,Wlttot_Lasswell,Wlbgain_Lasswell,Wlbloss_Lasswell,Wlbphys_Lasswell,Wlbpsyc_Lasswell,Wlbpt_Lasswell,Wlbtot_Lasswell,Enlgain_Lasswell,Enlloss_Lasswell,Enlends_Lasswell,Enlpt_Lasswell,Enloth_Lasswell,Enltot_Lasswell,Sklasth_Lasswell,Sklpt_Lasswell,Skloth_Lasswell,Skltot_Lasswell,Trngain_Lasswell,Trnloss_Lasswell,Tranlw_Lasswell,Meanslw_Lasswell,Endslw_Lasswell,Arenalw_Lasswell,Ptlw_Lasswell,Nation_Lasswell,Anomie_Lasswell,Negaff_Lasswell,Posaff_Lasswell,Surelw_Lasswell,If_Lasswell,Notlw_Lasswell,Timespc_Lasswell,formlw_Lasswell,Anger_NRC,Anticipation_NRC,Disgust_NRC,Fear_NRC,Joy_NRC,Negative_NRC,Positive_NRC,Sadness_NRC,Surprise_NRC,Trust_NRC,word_count
0,0,1,0,0,1,1,6,3,1,0,4,1,0,0,0,0,0,0,0,2,0,0,0,2,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,2,0,1,3,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,1,0,1,1,3,1,0,1,0,0,3,2,3,0,0,2,0,4,0,0,0,0,0,0,0,3,2,3,0,0,0,0,0,0,3,0,0,0,0,0,0,3,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,2,0,0,3,3,0,0,0,0,0,0,0,0,0,57,10
1,0,5,0,0,5,1,0,0,4,1,4,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,3,0,0,1,0,0,0,1,0,4,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,2,0,0,2,0,5,0,0,0,0,0,0,0,5,2,4,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,2,0,0,0,0,3,3,0,0,0,0,0,0,2,0,3,0,0,0,0,0,0,0,1,0,1,2,0,0,0,1,0,0,0,0,0,44,10
2,13,5,13,12,5,4,10,2,0,3,22,7,3,1,0,4,5,5,1,8,9,2,4,1,0,1,7,1,0,3,2,0,3,4,1,2,7,0,0,0,0,0,7,0,4,4,0,0,0,0,0,8,4,0,0,0,2,2,0,0,2,0,0,0,2,2,2,1,0,1,0,4,0,0,1,0,0,0,0,4,0,0,4,0,0,0,0,0,4,0,1,1,0,4,0,0,0,1,0,3,0,3,0,0,4,0,4,1,0,0,0,0,0,1,13,12,5,0,0,0,1,0,0,0,3,1,1,0,1,0,7,1,0,0,0,0,1,0,0,0,0,4,0,0,6,10,1,0,0,1,0,0,3,0,0,3,0,0,0,0,6,6,2,0,2,4,0,0,4,2,3,0,2,0,0,3,0,5,6,0,4,0,1,2,1,1,4,2,4,4,2,82,17
3,0,0,0,1,0,0,2,0,0,0,5,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,5,0,0,0,0,0,0,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,3,1,3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,28,6
4,3,3,3,0,2,5,7,2,7,0,6,6,0,0,0,0,0,2,0,3,4,0,0,0,0,1,2,1,0,0,0,0,0,0,2,3,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,3,3,0,0,0,0,0,2,1,1,0,1,0,0,0,1,2,0,1,0,0,0,1,0,1,0,0,1,1,0,0,1,2,1,0,0,0,0,2,0,0,0,8,0,0,0,0,0,4,0,6,0,0,1,0,2,0,

In [ ]:

matches

{'Positiv_GI': {'pass': 1,
  'dote': 1,
  'positive': 1,
  'brave': 1,
  'rave': 1,
  'gentle': 1,
  'fellow': 1,
  'luck': 1,
  'lucky': 1,
  'rally': 1,
  'attract': 1,
  'attractive': 1,
  'round': 1,
  'stud': 1,
  'discuss': 1,
  'important': 1,
  'appreciate': 1,
  'genuine': 1,
  'sage': 1,
  'train': 1,
  'agility': 1,
  'motivate': 1,
  'motivated': 1,
  'agile': 1,
  'live': 1,
  'eye': 1,
  'bonus': 1,
  'human': 1,
  'wonder': 1,
  'wonderful': 1,
  'stand': 1,
  'amazing': 1,
  'main': 1,
  'kid': 1,
  'ball': 2,
  'back': 2,
  'fun': 2,
  'game': 2,
  'pro': 2,
  'award': 2,
  'marry': 2,
  'actual': 2,
  'hit': 2,
  'good': 2,
  'give': 2,
  'interest': 3,
  'help': 3,
  'content': 3,
  'respect': 3,
  'great': 3,
  'share': 3,
  'glad': 4,
  'welcome': 4,
  'make': 4,
  'know': 4,
  'better': 4,
  'well': 4,
  'real': 4,
  'exact': 4,
  'care': 4,
  'worth': 4,
  'ally': 5,
  'clear': 5,
  'gain': 6,
  'even': 6,
  'art': 8,
  'happy': 8,
  'right': 8,
  'like': 8,
  'p

In [ ]:
features['docs']=docs
features

,Positiv_GI,Negativ_GI,Pstv_GI,Affil_GI,Ngtv_GI,Hostile_GI,Strong_GI,Power_GI,Weak_GI,Submit_GI,Active_GI,Passive_GI,Pleasur_GI,Pain_GI,Feel_GI,Arousal_GI,Emot_GI,Virtue_GI,Vice_GI,Ovrst_GI,Undrst_GI,Academ_GI,Doctrin_GI,Econ_2_GI,Exch_GI,Econ_GI,Exprsv_GI,Legal_GI,Milit_GI,Polit_2_GI,Polit_GI,Relig_GI,Role_GI,Coll_GI,Work_GI,Ritual_GI,Socrel_GI,Race_GI,Kin_2_GI,Male_GI,Female_GI,Nonadlt_GI,Hu_GI,Ani_GI,Place_GI,Social_GI,Region_GI,Route_GI,Aquatic_GI,Land_GI,Sky_GI,Object_GI,Tool_GI,Food_GI,Vehicle_GI,Bldgpt_GI,Comnobj_GI,Natobj_GI,Bodypt_GI,Comform_GI,Com_GI,Say_GI,Need_GI,Goal_GI,Try_GI,Means_GI,Persist_GI,Complet_GI,Fail_GI,Natrpro_GI,Begin_GI,Vary_GI,Increas_GI,Decreas_GI,Finish_GI,Stay_GI,Rise_GI,Exert_GI,Fetch_GI,Travel_GI,Fall_GI,Think_GI,Know_GI,Causal_GI,Ought_GI,Perceiv_GI,Compare_GI,Eval_2_GI,Eval_GI,Solve_GI,Abs_2_GI,Abs_GI,Quality_GI,Quan_GI,Numb_GI,Ord_GI,Card_GI,Freq_GI,Dist_GI,Time_2_GI,Time_GI,Space_GI,Pos_GI,Dim_GI,Rel_GI,Color_GI,Self_GI,Our_GI,You_GI,Name_GI,Yes_GI,No_GI,Negate_GI,Intrj_GI,Iav_GI,Dav_GI,Sv_GI,Ipadj_GI,Indadj_GI,Powgain_Lasswell,Powloss_Lasswell,Powends_Lasswell,Powaren_Lasswell,Powcon_Lasswell,Powcoop_Lasswell,Powaupt_Lasswell,Powpt_Lasswell,Powdoct_Lasswell,Powauth_Lasswell,Powoth_Lasswell,Powtot_Lasswell,Rcethic_Lasswell,Rcrelig_Lasswell,Rcgain_Lasswell,Rcloss_Lasswell,Rcends_Lasswell,Rctot_Lasswell,Rspgain_Lasswell,Rsploss_Lasswell,Rspoth_Lasswell,Rsptot_Lasswell,Affgain_Lasswell,Affloss_Lasswell,Affpt_Lasswell,Affoth_Lasswell,Afftot_Lasswell,Wltpt_Lasswell,Wlttran_Lasswell,Wltoth_Lasswell,Wlttot_Lasswell,Wlbgain_Lasswell,Wlbloss_Lasswell,Wlbphys_Lasswell,Wlbpsyc_Lasswell,Wlbpt_Lasswell,Wlbtot_Lasswell,Enlgain_Lasswell,Enlloss_Lasswell,Enlends_Lasswell,Enlpt_Lasswell,Enloth_Lasswell,Enltot_Lasswell,Sklasth_Lasswell,Sklpt_Lasswell,Skloth_Lasswell,Skltot_Lasswell,Trngain_Lasswell,Trnloss_Lasswell,Tranlw_Lasswell,Meanslw_Lasswell,Endslw_Lasswell,Arenalw_Lasswell,Ptlw_Lasswell,Nation_Lasswell,Anomie_Lasswell,Negaff_Lasswell,Posaff_Lasswell,Surelw_Lasswell,If_Lasswell,Notlw_Lasswell,Timespc_Lasswell,formlw_Lasswell,Anger_NRC,Anticipation_NRC,Disgust_NRC,Fear_NRC,Joy_NRC,Negative_NRC,Positive_NRC,Sadness_NRC,Surprise_NRC,Trust_NRC,word_count,docs
0,0,1,0,0,1,1,6,3,1,0,4,1,0,0,0,0,0,0,0,2,0,0,0,2,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,2,0,1,3,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,1,0,1,1,3,1,0,1,0,0,3,2,3,0,0,2,0,4,0,0,0,0,0,0,0,3,2,3,0,0,0,0,0,0,3,0,0,0,0,0,0,3,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,2,0,0,3,3,0,0,0,0,0,0,0,0,0,57,10,Didn't the Mets have a .909 win percentage at ...
1,0,5,0,0,5,1,0,0,4,1,4,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,3,0,0,1,0,0,0,1,0,4,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,2,0,0,2,0,5,0,0,0,0,0,0,0,5,2,4,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,2,0,0,0,0,3,3,0,0,0,0,0,0,2,0,3,0,0,0,0,0,0,0,1,0,1,2,0,0,0,1,0,0,0,0,0,44,10,"I could change [NAME], she need me in her life"
2,13,5,13,12,5,4,10,2,0,3,22,7,3,1,0,4,5,5,1,8,9,2,4,1,0,1,7,1,0,3,2,0,3,4,1,2,7,0,0,0,0,0,7,0,4,4,0,0,0,0,0,8,4,0,0,0,2,2,0,0,2,0,0,0,2,2,2,1,0,1,0,4,0,0,1,0,0,0,0,4,0,0,4,0,0,0,0,0,4,0,1,1,0,4,0,0,0,1,0,3,0,3,0,0,4,0,4,1,0,0,0,0,0,1,13,12,5,0,0,0,1,0,0,0,3,1,1,0,1,0,7,1,0,0,0,0,1,0,0,0,0,4,0,0,6,10,1,0,0,1,0,0,3,0,0,3,0,0,0,0,6,6,2,0,2,4,0,0,4,2,3,0,2,0,0,3,0,5,6,0,4,0,1,2,1,1,4,2,4,4,2,82,17,True i hate going to parties or concerts but l...
3,0,0,0,1,0,0,2,0,0,0,5,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,5,0,0,0,0,0,0,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,3,1,3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,28,6,Because that sub is for VLs.
4,3,3,3,0,2,5,7,2,7,0,6,6,0,0,0,0,0